In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# import package to unzip .7zip file
!sudo apt-get install -y p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-3+deb9u1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [4]:
# locally unzip the audio files in train.7z
# !cd ../Data/ && p7zip -d train.7z

In [1]:
%matplotlib inline

import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import torch
from torch.utils.data import DataLoader
from torch import optim

from utils import log_textfile, load_model
from basetrainer import BaseTrainer
from dataloader import load_data
from CustomDataset import CustomDatasetSimple
from models.resnet import ResNet, resnet34
from attacks import fgsm, pgd_linf

In [6]:
DATADIR = '../Data/'

trainset, valset = load_data(DATADIR)

There are 21105 train and 2577 val samples


In [7]:
trainset_dl = trainset
valset_dl = valset
#test_dl = testset_sub

train_filepaths = [i[2] for i in trainset_dl]
train_labels = [i[0] for i in trainset_dl]
valid_filepaths = [i[2] for i in valset_dl]
val_labels = [i[0] for i in valset_dl]
#test_filepaths = [i[2] for i in test_dl]
#test_labels = [i[0] for i in valset_dl]

In [8]:
rate, data = wavfile.read(train_filepaths[0])

In [9]:
# from utils import get_mean_std
#mean, std = get_mean_std(train_data_loader)

mean=-3.1259581955996425
std=0.8961027914827521
batch_size=32
num_workers=8
data_train_sub = CustomDatasetSimple(train_filepaths, train_labels, mean, std)
data_valid_sub = CustomDatasetSimple(valid_filepaths, val_labels, mean, std)

train_data_loader = torch.utils.data.DataLoader(
    dataset=data_train_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

valid_data_loader = torch.utils.data.DataLoader(
    dataset=data_valid_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

In [10]:
alpha_range = [0.1, 0.05, 0.01]

for alpha in alpha_range:
    MODELNAME = 'pgd_alpha_' + str(alpha)
    LOGFILE_PATH = 'logs/' + MODELNAME

    model = resnet34(pretrained=False, progress=False).cuda()

    criterion = torch.nn.CrossEntropyLoss()

    for param in model.parameters():
      param.requires_grad = True

    trainer_pgd = BaseTrainer(model=model, 
                          train_dl=train_data_loader, 
                          valid_dl=valid_data_loader, 
                          criterion=criterion, 
                          model_filename=MODELNAME, 
                          n_epochs=3)

    trainer_pgd.fit_model_new(optimizer=optim.Adam(model.parameters(), lr=.001), 
                          n_epochs=1, 
                          LOGFILE_PATH=LOGFILE_PATH,
                          model_filename=MODELNAME, 
                          attack=None, 
                          epsilon=0.3, 
                          alpha=alpha, 
                          num_iter=7)
    del(trainer_pgd)

20
[epoch: 1, batch:     20] loss: 0.07019 time model: 0.02197 acc: 0.19375
40
[epoch: 1, batch:     40] loss: 0.06671 time model: 0.02124 acc: 0.20469
60
[epoch: 1, batch:     60] loss: 0.06422 time model: 0.02100 acc: 0.23646
80
[epoch: 1, batch:     80] loss: 0.06324 time model: 0.02087 acc: 0.24766
100
[epoch: 1, batch:    100] loss: 0.06248 time model: 0.02079 acc: 0.26750
120
[epoch: 1, batch:    120] loss: 0.06161 time model: 0.02073 acc: 0.28021
140
[epoch: 1, batch:    140] loss: 0.06084 time model: 0.02069 acc: 0.29286
160
[epoch: 1, batch:    160] loss: 0.05984 time model: 0.02066 acc: 0.30449
180
[epoch: 1, batch:    180] loss: 0.05880 time model: 0.02063 acc: 0.31528
200
[epoch: 1, batch:    200] loss: 0.05769 time model: 0.02062 acc: 0.32906
220
[epoch: 1, batch:    220] loss: 0.05658 time model: 0.02060 acc: 0.34403
240
[epoch: 1, batch:    240] loss: 0.05535 time model: 0.02059 acc: 0.35768
260
[epoch: 1, batch:    260] loss: 0.05466 time model: 0.02058 acc: 0.36659
280

RuntimeError: CUDA out of memory. Tried to allocate 28.00 MiB (GPU 0; 11.17 GiB total capacity; 3.94 GiB already allocated; 22.94 MiB free; 53.57 MiB cached)